# Julia: Crash Course and Solving ODEs

## Contents
1. [Julia Crash Course](#julia)
2. [DifferentialEquations.jl](#diffeq)

## 1. Julia Crash Course <a class="anchor" id="julia"></a>


### 1.1. Why Julia

- Historically, we might divide programming languages commonly used for scientific computing into so-called "high-level" and "low-level" languages:


| Language Type | Implementation | Type System | Syntax | Speed | Examples
| :- | :- | :- | :- | :- | :- |
| High-level | [Interpreted](https://en.wikipedia.org/wiki/Interpreter_(computing)) | [Dynamic](https://en.wikipedia.org/wiki/Type_system#Dynamic_type_checking_and_runtime_type_information) | Easy, modern | Painfully slow | Python, R, Mathematica
| Low-level | [Compiled](https://en.wikipedia.org/wiki/Compiled_language) | [Static](https://en.wikipedia.org/wiki/Type_system#Static_type_checking) | Hard, obscure | Blazing fast | Fortran, C, C++

- This divide gives rise to the dreaded [**two-language problem**](https://www.nature.com/articles/d41586-019-02310-3): scientific programmes are often prototyped in a high-level language, such as Python, but then have to be translated into a low-level language, such as C, when speed becomes important. If you've ever had to convert a Python for-loop into vectorised NumPy code, then you've already experienced the two-language problem yourself.
<br>

- Julia aims to solve the two-language problem, giving scientists the flexibility to write code with the ease of Python while executing with the speed of Fortran or C, all inside one language.
<br>

- That is to say, Julia offers the best of both worlds:
    - it's ([JIT](https://en.wikipedia.org/wiki/Just-in-time_compilation)) **compiled**,
    - it's **dynamically typed**,
    - it has a clean **modern syntax**,
    - it's **exceptionally** [**fast**](https://julialang.org/benchmarks/).
    

- TODO: Add some examples of high profile uses of Julia, e.g. Clima.
- TODO: Add something about the REPL between here and the next section.

### 1.2. Arrays

- Unsurprisingly, given its emphasis on technical computing, Julia has a first-class array implementation, `Array`.
<br>

- It's a good idea to read the array [documentation](https://docs.julialang.org/en/v1/manual/arrays/), but here we will give a very brief overview of what you need to get started.

#### 1.2.1. `Array` can have arbitrary dimensions

- In Julia, arrays can have **arbitrary dimensions**. You can think of an array as a finite n-dimensional grid, with an object placed at each gridpoint.

In [6]:
a = zeros(2, 3, 4)  # zeros is a useful constructor
typeof(a)

Array{Float64, 3}

In [3]:
a = zeros(2, 3)
typeof(a)

Matrix{Float64} (alias for Array{Float64, 2})

In [4]:
a = zeros(2)
typeof(a)

Vector{Float64} (alias for Array{Float64, 1})

In [7]:
a = zeros()
typeof(a)

Array{Float64, 0}

#### 1.2.2. `Array` is 1-indexed

- Unlike other programming languages you might be familiar with, Julia arrays are 1-indexed rather than 0-indexed.

In [11]:
a = rand(3)

3-element Vector{Float64}:
 0.5301631387140582
 0.5518406542346757
 0.310180193598278

In [12]:
a[0]  # Attempt to get the element at position 0 of a

LoadError: BoundsError: attempt to access 3-element Vector{Float64} at index [0]

In [14]:
a[1]  # Get the first element of a

0.5301631387140582

#### 1.2.3. More General Array Indexing

- The general syntax for indexing into an n-dimensional array `A` is:

    `A[I_1, I_2, ..., I_n]`,

    where each `I_k` may be a scalar integer, an array of integers, or any other supported index (see the [docs]((https://docs.julialang.org/en/v1/manual/arrays/#man-supported-index-types)) for other supported indices).

In [28]:
a = rand(2, 3)

2×3 Matrix{Float64}:
 0.185933  0.0478174  0.481766
 0.371688  0.972404   0.725433

In [29]:
a[1, 2]

0.04781739072882396

In [30]:
a[2, 1:2]  # 1:2 is a range object

2-element Vector{Float64}:
 0.3716881178129522
 0.9724037155246469

#### 1.2.4. Indexed Assignment
- As you might expect, arrays are mutable.
<br>

- Similar to array indexing, the general syntax for assigning values in an n-dimensional array `A` is:

    `A[I_1, I_2, ..., I_n] = X`,

    where each `I_k` may be a scalar integer, an array of integers, or any other supported index.

In [31]:
a[1, 2] = 2
a

2×3 Matrix{Float64}:
 0.185933  2.0       0.481766
 0.371688  0.972404  0.725433

- If any index `I_k` is itself an array, then the right hand side `X` must also be an array with the same shape as the result of indexing `A[I_1, I_2, ..., I_n]` or a vector with the same number of elements. 

In [32]:
size(a[1, [1,2]])  # Two-element vector

(2,)

In [33]:
size([3, 4])  # Also a two-element vector

(2,)

In [34]:
a[2, [1,2]] = [3, 4]  # Which means we can do this
a

2×3 Matrix{Float64}:
 0.185933  2.0  0.481766
 3.0       4.0  0.725433

- Finally, the element-wise assignment operator `.=` may be used to broadcast `X` across the selected locations:

    `A[I_1, I_2, ..., I_n] .= X`

In [36]:
a[:, 3] .= 5.0
a

2×3 Matrix{Float64}:
 0.185933  2.0  5.0
 3.0       4.0  5.0

### 1.3. Functions, Methods, and Multiple Dispatch

- At a high level, functions in Julia are conceputally similar to mathematical functions: they map inputs to return values.

#### 1.3.1. Standard Function Definition Syntax 

- This is probably the form you'll see and use most often.

In [37]:
function f(x, y)
    return x + y
end

f (generic function with 1 method)

In [38]:
f(1, 2)

3

- Note that the `return` keyword is optional, since by default a function will return the value of the **last expression evaluated**. However, I generally prefer to be as explicit as possible so I include it.
<br>

- By convention, functions which modify their arguments in-place have names ending in `!`.

In [43]:
function square_array!(a)
    a .= a.^2
    return nothing
end

square_array! (generic function with 1 method)

In [59]:
a = rand(2, 3)

2×3 Matrix{Float64}:
 0.243873  0.260621  0.465913
 0.262262  0.426214  0.0256367

In [60]:
square_array!(a)
a

2×3 Matrix{Float64}:
 0.0594738  0.0679234  0.217075
 0.0687815  0.181658   0.000657241

- For more information on defining functions, such as [optional arguments](https://docs.julialang.org/en/v1/manual/functions/#Optional-Arguments) and [keyword arguments](https://docs.julialang.org/en/v1/manual/functions/#Keyword-Arguments), see the [docs](https://docs.julialang.org/en/v1/manual/functions/).

#### 1.3.2. Methods

- Closely related to functions is the idea of a method.
<br>

- A method defines one possible behaviour for a function, _given the_ number _and_ types _of its arguments_.


In [62]:
function add(x, y)
    return x + y
end

function add(x, y, z)
    return x + y + z
end

add (generic function with 2 methods)

In [63]:
methods(add)

# 2 methods for generic function "add":
[1] add(x, y) in Main at In[62]:1
[2] add(x, y, z) in Main at In[62]:5

In [67]:
add(1, 2)

3

In [66]:
add(1, 2, 3)

6

- Conceptually, we have one function (addition), with many possible behaviours, depending on the types of the inputs (integers, floats, matrices, and so on).
<br>

- We can use _type annotations_ to make our method definitions more specific. 

In [64]:
function add(X::Array, Y::Array)
    return X .+ Y
end

add (generic function with 3 methods)

In [68]:
methods(add)

# 3 methods for generic function "add":
[1] add(X::Array, Y::Array) in Main at In[64]:1
[2] add(x, y) in Main at In[62]:1
[3] add(x, y, z) in Main at In[62]:5

In [78]:
A = rand(2, 3)
B = rand(2, 3)
add(A, B)

2×3 Matrix{Float64}:
 1.70928   0.636454  1.11681
 0.537059  0.367346  0.868063

#### 1.3.3. Multiple Dispatch

- Given a single function with many methods, how does Julia choose which method to call?

> The choice of which method to execute when a function is applied is called dispatch. **Julia allows the dispatch process to choose which of a function's methods to call based on the number of arguments given, and on the types of all of the function's arguments**. This is different than traditional object-oriented languages, where dispatch occurs based only on the first argument, which often has a special argument syntax, and is sometimes implied rather than explicitly written as an argument. **Using all of a function's arguments to choose which method should be invoked, rather than just the first, is known as multiple dispatch**. 


- In particular, Julia will automatically select the **most specific** method matching the arguments provided.
<br>

- _Multiple dispatch is the core programming paradigm of the Julia language_.

### 1.4. Structs (or: Julia Objects)

- The most common user-defined type in Julia is a **composite type** or `struct`. This is the Julia equivalent of a `Class` in Python.

#### 1.4.1. Defining Structs

- Composite types are introduced with the `struct` keyword followed by a block of field names.

In [71]:
struct Foo
    bar
    baz
end

#### 1.4.2. Instantiating Structs

- To create a new object of a given type, simply apply the type name like a function.

In [72]:
foo = Foo("Machine", "Learning")

Foo("Machine", "Learning")

In [73]:
typeof(foo)

Foo

#### 1.4.3. Accessing the Fields of a Struct

- You can access the field names of a composite type using the standard `foo.bar` notation.

In [75]:
foo.bar

"Machine"

In [76]:
foo.baz

"Learning"

#### 1.4.4 Structs are Not Mutable

- Composite objects declared with `struct` are **immutable**; they cannot be modified after construction.

In [77]:
foo.baz = "Intelligence"

LoadError: setfield!: immutable struct of type Foo cannot be changed

- **N.B.** An immutable object might contain mutable objects, such as arrays, as fields. Those mutable field values will remain mutable; only the fields of the immutable object itself cannot be changed to point to different objects.

In [79]:
struct ArrayStruct
    a::Array
end

array_struct = ArrayStruct([1.0, 2.0])
array_struct.a

2-element Vector{Float64}:
 1.0
 2.0

In [80]:
# Modify the mutable object
array_struct.a[1] = 3.0
array_struct.a

2-element Vector{Float64}:
 3.0
 2.0

In [81]:
# Attempt to modify the field value itself
array_struct.a = [3.0, 4.0]

LoadError: setfield!: immutable struct of type ArrayStruct cannot be changed

- Why is it like this? Isn't this unnecessarily restrictive for the programmer?
<br>

- The main reason for structs to be immutable is to allow the compiler to effectively optimise your code. In particular, it allows your objects to be stored efficiently in memory.
<br>

- If you _really_ do need a mutable struct (you probably don't), you can simply declare one with the `mutable` keyword:

In [82]:
mutable struct MyMutableStruct
    a
end
m = MyMutableStruct("original value")
m.a

"original value"

In [83]:
m.a = "new value"
m.a

"new value"

### 1.5. Control Flow

#### 1.5.1 The `for` Loop

- The most common loop in Julia is the `for` loop:

In [84]:
for i = 1:5
    println(i)
end

1
2
3
4
5


- You will often find yourself using range objects together with `for` loops. This is a very useful construct for generating sequences of numbers and takes the general form:

    ```julia
    a:b:c
    ```

    where `a` is the starting number, `c` is the ending number, and `b` is the stepsize. By default `b = 1`.    


- **N.B.** ranges in Julia are **inclusive**, i.e. the range includes `c`.
<br>
- The `for` loop can also iterate over all the items of a container (e.g. `Array`, `Tuple`), not just a range object:

In [85]:
stuff = ["Machine", "Learning"]
for thing in stuff
    println(thing)
end

Machine
Learning


#### 1.5.2. Conditional Evaluation

- The standard `if`-`elseif`-`else` syntax in Julia is:

```julia
if x < y
    println("x is less than y")
elseif x > y
    println("x is greater than y")
else
    println("x is equal to y")
end
```

- Unlike other languages (e.g. Python), the value given to the conditional expression __must be a boolean__, i.e. `true` or `false`. In other words, there are no "truthy" and "falsy" values in Julia.

### 1.6. Pkg.jl - The Julia Package Manager

- [Pkg.jl](https://pkgdocs.julialang.org/v1/) is Julia's built-in package manager.
<br>

- It's comparable to the combination of `pip` and `virtualenv`, or `conda`, in Python. Except it actually works.
<br>

- There are two ways to use Pkg.jl:
    1. [The Pkg REPL](https://pkgdocs.julialang.org/v1/repl/), which you can enter by simply typing `]` at the Julia REPL.
    2. [The Pkg API](https://pkgdocs.julialang.org/v1/api/), useful when you want to interact with the package manager from a script or Jupyter notebook.
<br><br>

- Pkg.jl is built around the idea of environments, and an environment is defined by a `Project.toml` file.
<br>

#### 1.6.1. Activating an Environment

#### 1.6.2. Instantiating an Environment

#### 1.6.3. Adding Packages

## 2. DifferentialEquations.jl <a class="anchor" id="diffeq"></a>

- [DifferentialEquations.jl](https://diffeq.sciml.ai/stable/) is a comprehensive suite of algorithms for the numerical solution of differential equations in Julia.
<br>

- First, let's install the package:

In [93]:
using Pkg
Pkg.activate(".")
Pkg.add("DifferentialEquations")

  Activating project at `~/code/ML-DS-Workshop-23`
   Resolving package versions...
  No Changes to `~/code/ML-DS-Workshop-23/Project.toml`
  No Changes to `~/code/ML-DS-Workshop-23/Manifest.toml`


- Once the package is installed, we need to import it. This is most commonly done with the `using` keyword, although see [here](https://docs.julialang.org/en/v1/manual/code-loading/) for other options and the full details of code loading in Julia.

In [94]:
using DifferentialEquations

### 2.1. Solving ODEs

- With all of this done, solving an ODE in Julia is a simple 3-step process.

#### Step 1: Define the RHS of your ODE

- For a standard ODE of the form $\dot u = f(u, t; p)$, $u \in \mathbb R^n$, we need to define $f$.

In [ ]:
# Lorenz '63 (OOP)
function f(u, p, t)
    x, y, z = u
    σ, β, ρ = p
    
    dx = σ * (y - x)
    dy = x * (ρ - z) - y
    dz = x * y - β * z
    
    return [dx, dy, dz]
end

#### Step 2: Set up the ODE Problem

In [ ]:
u0 = rand(3)             # Initial conditions
tspan = (0.0, 10.0)      # Timespan of the integration
p = [10.0, 8 / 3, 28.0]  # Parameters (optional)

prob = ODEProblem(f, u0, tspan, p)

#### Step 3: Solve the ODE Problem

In [ ]:
sol = solve(prob)

In [ ]:
Array(sol)

#### Alternative: In-Place Updates

- So far we've defined and solved our problem in the out-of-place (OOP) form.
<br>

- Often it is more efficient to define the problem in the in-place (IIP) form.

In [95]:
# Lorenz '63 (IIP)
function f!(du, u, p, t)
    x, y, z = u
    σ, β, ρ = p
    
    du[1] = σ * (y - x)
    du[2] = x * (ρ - z) - y
    du[3] = x * y - β * z
    
    return nothing
end

f! (generic function with 1 method)

In [ ]:
using BenchmarkTools
@btime solve(prob)  # experiment with this yourself

### 2.2. Plotting the Solution

- DifferentialEquations.jl provides recipes for plotting ODE solutions using the Plots.jl package.
<br>

- First, make sure the Plots.jl package is installed and imported.

In [96]:
Pkg.add("Plots")
using Plots

   Resolving package versions...
    Updating `~/code/ML-DS-Workshop-23/Project.toml`
  [91a5bcdd] + Plots v1.38.5
    Updating `~/code/ML-DS-Workshop-23/Manifest.toml`
  [d1d4a3ce] + BitFlags v0.1.7
  [944b1d66] + CodecZlib v0.7.1
  [35d6a980] + ColorSchemes v3.20.0
  [3da002f7] + ColorTypes v0.11.4
  [c3611d14] + ColorVectorSpace v0.9.10
  [5ae59095] + Colors v0.12.10
  [d38c429a] + Contour v0.6.2
  [c87230d0] + FFMPEG v0.4.1
  [53c48c17] + FixedPointNumbers v0.8.4
  [59287772] + Formatting v0.4.2
  [28b8d3ca] + GR v0.71.7
  [42e2da0e] + Grisu v1.0.2
  [cd3eb016] + HTTP v1.7.4
  [83e8ac13] + IniFile v0.5.1
  [1019f520] + JLFzf v0.1.5
  [682c06a0] + JSON v0.21.3
  [b964fa9f] + LaTeXStrings v1.3.0
  [23fbe1c1] + Latexify v0.15.18
  [e6f89c97] + LoggingExtras v1.0.0
  [739be429] + MbedTLS v1.1.7
  [442fdcdd] + Measures v0.3.2
  [4d8831e6] + OpenSSL v1.3.3
  [69de0a69] + Parsers v2.5.7
  [b98c9c47] + Pipe v1.3.0
  [ccf2f8ad] + PlotThemes v3.1.0
  [995b91a9] + PlotUtils v1.3.4
  [91a5bcdd

- Then, plotting trajectories is as simple as calling `plot` with the solution object.

In [ ]:
plot(sol)

### 2.3. Configuring the Solver: Common Solver Options

- When we solved our ODE problem just now, we simply called `solve(prob)` and allowed DifferentialEquations.jl to choose sensible defaults for the ODE solver algorithm and various other important parameters.
<br>

- In practice, we will often want to specify the precise behaviour of the solver.


#### 2.3.1. Specifying the Solver Algorithm

- A huge number of [ODE solvers](https://diffeq.sciml.ai/stable/solvers/ode_solve/) are implemented (or interfaced) in DifferentialEquations.jl.
<br>

- To specify a specific algorithm, simply pass it as the second argument to `solve`.

In [ ]:
solve(prob, Tsit5())

#### 2.3.2. Save Times

- By default, an adaptive step size algorithm will simply save the result at the times where it stopped.
<br>

- Often, you'll want the solver to output the solution at predefined times. To do this, use the keyword argument `saveat`.

In [ ]:
sol = solve(prob, Tsit5(), saveat = 0.1)

- This does **not** force the adaptive solver to stop at those times. Instead, the solution at intermediate times is calculated using an efficient interpolation function which, it can be shown, preserves the order of the solution.

#### 2.3.3. Error Tolerances

- As you saw earlier, adaptive step size algorithms choose the step size such that the estimated local error at each step remains below some user-defined tolerance.
<br>

- In general, it is required that the normalised error estimate satisfies componentwise

$$\mathrm{err_i} \leq \mathrm{abstol_i} + \max(|y_{0i}|, |y_{1i}|) \cdot \mathrm{reltol_i},$$

where $|y_{0i}|$ is the initial value, $|y_{1i}|$ is the solver's estimate of the solution, $\mathrm{abstol_i}$ is the *absolute tolerance* and $\mathrm{reltol_i}$ is the *relative tolerance*.
<br>

- The relative tolerance scales with the solution, so you can interpret a relative tolerance of `1e-3` as meaning, roughly, that the solution is (locally) correct to 3 digits.
<br>

- On the other hand, the absolute tolerance controls the (local) error of the solution around zero.
<br>

- Control the stepping behaviour with the `abstol` and `reltol` keyword arguments to `solve`.

In [ ]:
solve(prob, Tsit5(), reltol = 1e-6, abstol = 1e-6)